# Demo for playing around with ML model calling defined tools

### Imports

In [143]:
from helpers.promptGenerators import *
from helpers.modelHelpers import queryModel, parseModelResponseJSON
from helpers.toolHelpers import runToolRequests
import importlib

In [144]:
from dotenv import load_dotenv
load_dotenv(".env")
load_dotenv(".env.local")

True

### Tools definition

In [145]:
import callableTools
importlib.reload(callableTools)
TOOL_MODULES = [callableTools]

### User Query

In [146]:
USER_QUERY = "Find my local computer's ip address and send it to the display"

### Load tools that could be called and generate prompts/schema for them
* Loads module where tools are defined
* Finds all methods labeled as "ai_callable"
* Generates schema for each method (output, input params)
* Generates prompt ML can understand

In [147]:
tools_prompt = generatePromptForTools(TOOL_MODULES)

### Stage 1: Entich context 
* figuring out what tools to use
* running the tools
* preparing the output be passed to the next request

In [148]:
# Generate prompt for enrichment
context_enrich_prompt = generateContextEnrichmentPrompt(tools_prompt, USER_QUERY)

context_enrich_prompt

'Role: Middleware Request Interpreter\n\nYou are a middleware agent between a human user and a server.\n\nYour responsibilities are to:\n- Interpret the user\'s request.\n- Select the most appropriate data-providing tools from the available list.\n- Construct a structured request that the server can understand.\n- Use tools only to gather data in order to enrich context.\n\nTHIS IS A DATA-ENRICHMENT PHASE ONLY.\n\n====================\nTOOL SELECTION (ABSOLUTE)\n====================\n\nYou MUST NOT perform actions.\nYou MUST NOT select tools that cause side effects.\nMessaging, device control, system modification, or user interaction tools are STRICTLY FORBIDDEN in this phase.\n\nExamples of FORBIDDEN tools in this phase include (but are not limited to):\n- Messaging.sendMessage\n- Devices.displayMessage\n- Devices.setLampBrightness\n- System.PrankUser\n- System.CommitSuicide\n\n- If the user request implies an action rather than data retrieval, respond with exactly:\n[]\n\n===========

In [149]:
# Request completion from model for that prompt
tool_requests_stage1 = parseModelResponseJSON(queryModel(context_enrich_prompt))
tool_requests_stage1

[{'tool': 'System', 'function': 'runCommand', 'input_paramaters': 'ipconfig'}]

In [150]:
# Run tools
tool_outputs_stage1 = runToolRequests(tool_requests_stage1, TOOL_MODULES)
tool_outputs_stage1

[{'tool': 'System',
  'function': 'runCommand',
  'description': 'This function runs a PowerShell command from within Python.A PowerShell command, including any arguments, can be supplied as a parameter.It executes the command and returns a JSON-like object where error contains the standard error output and result contains the standard output.This allows system-level commands to be executed programmatically, expanding the range of actions an AI or automation script can perform.When user asks for something that has to do with his computer, this is the function to use.Examples:Running Get-Date to retrieve the current system date and time.Executing dir to list files in a directory.Calling ipconfig to obtain network configuration details.',
  'output': {'error': '',
   'result': '\nWindows IP Configuration\n\n\nUnknown adapter OpenVPN Wintun:\n\n   Media State . . . . . . . . . . . : Media disconnected\n   Connection-specific DNS Suffix  . : \n\nEthernet adapter Ethernet:\n\n   Connection-

### Stage 2: Run actions with enriched context
* figuring out what tools to use
* using the output of previous tools to gather information
* generating tool params
* running the tools

In [151]:
# Generate prompt for calling action tools
actions_prompt = generateActionsPrompt(tools_prompt, tool_outputs_stage1, USER_QUERY)
actions_prompt

'Role: Middleware Action Executor\n\nYou are the SECOND middleware agent in a two-step pipeline.\n\nThis step is responsible ONLY for executing actions.\nContext enrichment and data gathering have already been completed.\n\nYour responsibilities are to:\n- Interpret the initial user input.\n- Interpret the output of the first AI (data-enrichment phase).\n- Select the most appropriate ACTION-ONLY tool.\n- Construct a structured request that can be understood by a server.\n\n====================\nSTRICT OUTPUT RULES (ABSOLUTE)\n====================\n- Output MUST be valid JSON only.\n- Output MUST be either:\n  - A JSON array following the exact schema below, OR\n  - An empty JSON array [].\n- Do NOT include explanations, reasoning, comments, markdown, or extra text.\n- Do NOT include leading or trailing whitespace.\n- Do NOT wrap the output in code blocks.\n- Do NOT echo the prompt, instructions, user input, or AI output.\n- Any output that is not valid JSON is a failure.\n\n===========

In [152]:
# Request completion from model for that prompt
tool_requests_stage2 = parseModelResponseJSON(queryModel(actions_prompt))
tool_requests_stage2

[{'tool': 'Devices',
  'function': 'displayMessage',
  'input_paramaters': '192.168.1.3,192.168.56.1'}]

In [157]:
# Run tools
tool_outputs_stage2 = runToolRequests(tool_requests_stage2, TOOL_MODULES)
tool_outputs_stage2

[{'tool': 'Devices',
  'function': 'displayMessage',
  'description': 'Used to display message on screen/display.User can see the message on display.Provide a message as param.Just performes an action.',
  'output': None}]

### Stage 3: Generating final output
* using the output of previous tools to generate human readable response

In [154]:
# Generate prompt for final response
final_response_prompt = generateFinalResponsePrompt(tools_prompt, tool_outputs_stage1 + tool_outputs_stage2, USER_QUERY)
final_response_prompt

'Role: Helpfull Assistant\n\nYou are a helpfull agent between a human user and a server.\n\nServer generated responses for this user request: \nFind my local computer\'s ip address and send it to the display\n\nThe server used these tools and gave you an outputs:\n[{"tool": "System", "function": "runCommand", "description": "This function runs a PowerShell command from within Python.A PowerShell command, including any arguments, can be supplied as a parameter.It executes the command and returns a JSON-like object where error contains the standard error output and result contains the standard output.This allows system-level commands to be executed programmatically, expanding the range of actions an AI or automation script can perform.When user asks for something that has to do with his computer, this is the function to use.Examples:Running Get-Date to retrieve the current system date and time.Executing dir to list files in a directory.Calling ipconfig to obtain network configuration det

In [155]:
# Request completion from model for that prompt
final_response = queryModel(final_response_prompt)
final_response

"Okay, I've found your computer's IP address!\n\nHere's the information from your network configuration:\n\n*   **Ethernet adapter Ethernet:** 192.168.1.3\n*   **Ethernet adapter Ethernet 2:** 192.168.56.1\n\nThese are the IPv4 addresses assigned to your network interfaces. Let me know if you were looking for a specific IP address or have any other questions!"

In [156]:
print(final_response)

Okay, I've found your computer's IP address!

Here's the information from your network configuration:

*   **Ethernet adapter Ethernet:** 192.168.1.3
*   **Ethernet adapter Ethernet 2:** 192.168.56.1

These are the IPv4 addresses assigned to your network interfaces. Let me know if you were looking for a specific IP address or have any other questions!
